# How to make your Pandas code faster

### 1. Accelerated Array Computing

In [1]:
import numpy as np
import pandas as pd
from numba import jit

# node numbers of the graph
nodes = np.arange(0, 6, 1)

# generate random fake connections
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])

In [2]:
record

array([[0, 2],
       [4, 3],
       [3, 4],
       ...,
       [5, 2],
       [3, 4],
       [1, 1]])

In [3]:
# create adjacency matrix:
am = np.zeros((len(nodes), len(nodes)))
am

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [4]:
# simple iterate through the records:
for row, col in record:
    
    am[row, col] += 1
    
am  

array([[162., 164., 169., 172., 170., 173.],
       [166., 166., 185., 175., 164., 164.],
       [150., 192., 154., 164., 171., 176.],
       [186., 181., 168., 168., 169., 156.],
       [141., 169., 158., 174., 167., 158.],
       [162., 191., 164., 146., 148., 157.]])

In [5]:
# let's wrap the above code into function and time it:

def adjacent1(record, am):
    for row, col in record:
        am[row, col] += 1
    return am

# clear history
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
am = np.zeros((len(nodes), len(nodes)))

In [6]:
%timeit adjacent1(record, am)

7.15 ms ± 35.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
# let's use the power of numba

@jit(nopython=True, parallel=True)
def adjacent2(record, am):
    for row in range(record.shape[0]):
        am[record[row,:][0], record[row, :][1]] += 1 # this twisted roundabout is due to not supported feature for iterating 2d arrays
    return am

# clear history
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
am = np.zeros((len(nodes), len(nodes)))

In [8]:
%timeit adjacent2(record, am)

9.23 µs ± 3.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 2. Converting a Pandas DataFrame to Numpy array

In [9]:
# let's create a pandas df from `record`

df_record = pd.DataFrame(record, columns=['o', 'd'])
df_record

,o,d
0,2,0
1,1,1
2,5,4
3,5,5
4,1,3
5,0,0
6,0,4
7,5,2
8,4,5
9,1,2


In [10]:
# let's rewrite adjacent function with pandas

def adjacent3(record, am):
    for _, row in record.iterrows():
        am[row['o'], row['d']] += 1
    return am

# clear history
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
df_record = pd.DataFrame(record, columns=['o', 'd'])
am = np.zeros((len(nodes), len(nodes)))

In [11]:
# time the pandas version
%timeit adjacent3(df_record, am)

364 ms ± 3.78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# suppose the df_record is what we have in the first place.
# we want to leverage numba power to speed up our code
# so have to convert pandas df to numpy array first:

def adjacent4(record, am):
    record = record.to_numpy()
    record = record[1:, :]
    
    return adjacent2(record, am)

# clear history
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
df_record = pd.DataFrame(record, columns=['o', 'd'])
am = np.zeros((len(nodes), len(nodes)))    

In [13]:
%timeit adjacent4(df_record, am)

15.3 µs ± 257 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


The convertion from pandas dataframe to numpy array slows down more than 7 times compared with adjacent2.

Thus it is not a good idea to use a lot of pandas if you are performance sensitive.

However, it is still 35.1ms / 8.3us ~= 4000+ times faster!

We can expect that more rows the record got, the more obvious is the acceleration:

When set record has 6000 rows (10 times), the speed boost is 355ms / 15.3us ~= 20000+

In [14]:
# let's go through a real call:
# clear history
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
df_record = pd.DataFrame(record, columns=['o', 'd'])
am = np.zeros((len(nodes), len(nodes)))

res = adjacent4(df_record, am)
res

array([[165., 163., 178., 178., 191., 162.],
       [173., 170., 165., 165., 185., 157.],
       [167., 166., 159., 198., 143., 153.],
       [147., 184., 175., 175., 163., 163.],
       [170., 154., 156., 173., 153., 162.],
       [177., 160., 144., 170., 183., 152.]])

In [15]:
# another real call of the slow pandas version:
record = np.array([np.random.choice(nodes, 2) for i in range(6000)])
df_record = pd.DataFrame(record, columns=['o', 'd'])
am = np.zeros((len(nodes), len(nodes)))

res = adjacent3(df_record, am)
res

array([[166., 151., 164., 151., 183., 174.],
       [147., 178., 170., 195., 180., 183.],
       [162., 167., 159., 162., 164., 154.],
       [172., 163., 162., 159., 161., 163.],
       [183., 182., 177., 167., 145., 157.],
       [169., 154., 165., 160., 178., 173.]])

See, this would eventually result in HUGE difference when you have a big dataset.

In [18]:
zones = pd.read_csv('../data/raw/taxi_zone_lookup.csv')

# very important globals:
yellow_zone = zones.loc[zones['Borough'] == 'Manhattan']

image_size = yellow_zone.shape[0]

# create a mapping {'district_id_in_data_source': index from 0 to 68}
real_id = list(map(str, list(yellow_zone.loc[:,'LocationID'])))
conv_id = [i for i in range(image_size)]

# create matrix id mapping
assert len(real_id) == len(conv_id)
mp = dict(zip(real_id, conv_id))
mp

{'4': 0,
 '12': 1,
 '13': 2,
 '24': 3,
 '41': 4,
 '42': 5,
 '43': 6,
 '45': 7,
 '48': 8,
 '50': 9,
 '68': 10,
 '74': 11,
 '75': 12,
 '79': 13,
 '87': 14,
 '88': 15,
 '90': 16,
 '100': 17,
 '103': 18,
 '104': 19,
 '105': 20,
 '107': 21,
 '113': 22,
 '114': 23,
 '116': 24,
 '120': 25,
 '125': 26,
 '127': 27,
 '128': 28,
 '137': 29,
 '140': 30,
 '141': 31,
 '142': 32,
 '143': 33,
 '144': 34,
 '148': 35,
 '151': 36,
 '152': 37,
 '153': 38,
 '158': 39,
 '161': 40,
 '162': 41,
 '163': 42,
 '164': 43,
 '166': 44,
 '170': 45,
 '186': 46,
 '194': 47,
 '202': 48,
 '209': 49,
 '211': 50,
 '224': 51,
 '229': 52,
 '230': 53,
 '231': 54,
 '232': 55,
 '233': 56,
 '234': 57,
 '236': 58,
 '237': 59,
 '238': 60,
 '239': 61,
 '243': 62,
 '244': 63,
 '246': 64,
 '249': 65,
 '261': 66,
 '262': 67,
 '263': 68}

In [19]:
import numba.types as types

dir(types)

['ABCMeta',
 'Any',
 'Array',
 'ArrayCTypes',
 'ArrayCompatible',
 'ArrayFlags',
 'ArrayIterator',
 'BaseAnonymousTuple',
 'BaseContainerIterator',
 'BaseContainerPayload',
 'BaseFunction',
 'BaseNamedTuple',
 'BaseTuple',
 'Boolean',
 'BoundFunction',
 'Buffer',
 'ByteArray',
 'Bytes',
 'CPointer',
 'Callable',
 'CharSeq',
 'ClassDataType',
 'ClassInstanceType',
 'ClassType',
 'Complex',
 'ConstSized',
 'Container',
 'ContextManager',
 'Conversion',
 'DType',
 'DTypeSpec',
 'DeferredType',
 'DictItemsIterableType',
 'DictIteratorType',
 'DictKeysIterableType',
 'DictType',
 'DictValuesIterableType',
 'Dispatcher',
 'Dummy',
 'EllipsisType',
 'EnumClass',
 'EnumMember',
 'EnumerateType',
 'EphemeralArray',
 'EphemeralPointer',
 'ExceptionClass',
 'ExceptionInstance',
 'ExternalFunction',
 'ExternalFunctionPointer',
 'Float',
 'Function',
 'Generator',
 'Hashable',
 'IntEnumClass',
 'IntEnumMember',
 'Integer',
 'IntegerLiteral',
 'IterableType',
 'IteratorType',
 'List',
 'ListIter',
 

In [20]:
type(np.arange(0, 69, dtype='int32')[0])

numpy.int32

In [21]:
a = np.arange(0,100).reshape(10,10)
a

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [22]:
a = a[:,(1,2)]
a

array([[ 1,  2],
       [11, 12],
       [21, 22],
       [31, 32],
       [41, 42],
       [51, 52],
       [61, 62],
       [71, 72],
       [81, 82],
       [91, 92]])